In [1]:
import pandas as pd
import numpy as np

In [2]:
import keras

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [3]:
data1 = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
data1.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
data1.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
data1.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [10]:
concrete_data_columns = data1.columns
predictors = data1[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = data1['Strength'] # Strength column
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [11]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [12]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [13]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

In [20]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))         
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

# build the model
model = regression_model()

# fit the model
epochs = 100
model.fit(X_train, y_train, epochs=epochs, verbose=2)

Epoch 1/100
 - 2s - loss: 1575.7386
Epoch 2/100
 - 1s - loss: 1554.9402
Epoch 3/100
 - 1s - loss: 1523.3654
Epoch 4/100
 - 1s - loss: 1468.6742
Epoch 5/100
 - 0s - loss: 1380.2610
Epoch 6/100
 - 0s - loss: 1244.3180
Epoch 7/100
 - 0s - loss: 1051.7622
Epoch 8/100
 - 0s - loss: 804.4417
Epoch 9/100
 - 0s - loss: 538.6881
Epoch 10/100
 - 0s - loss: 340.4230
Epoch 11/100
 - 0s - loss: 245.9356
Epoch 12/100
 - 0s - loss: 217.7079
Epoch 13/100
 - 0s - loss: 207.6951
Epoch 14/100
 - 0s - loss: 200.1561
Epoch 15/100
 - 0s - loss: 193.7251
Epoch 16/100
 - 0s - loss: 188.4055
Epoch 17/100
 - 0s - loss: 183.5439
Epoch 18/100
 - 0s - loss: 179.3236
Epoch 19/100
 - 0s - loss: 175.1104
Epoch 20/100
 - 0s - loss: 170.7506
Epoch 21/100
 - 0s - loss: 166.8477
Epoch 22/100
 - 0s - loss: 162.5372
Epoch 23/100
 - 0s - loss: 158.6176
Epoch 24/100
 - 0s - loss: 155.0127
Epoch 25/100
 - 0s - loss: 151.8008
Epoch 26/100
 - 0s - loss: 148.4751
Epoch 27/100
 - 0s - loss: 145.5216
Epoch 28/100
 - 0s - loss: 142

In [22]:
loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val

309/309 [==============================] - 0s 511us/step


70.95018701645934

In [23]:
from sklearn.metrics import mean_squared_error
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

70.95018740264115 0.0


In [24]:
total_mean_squared_errors = 50
epochs = 100
mean_squared_errors = []
for i in range(0, total_mean_squared_errors):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mean_squared_errors) + " mean squared errors with normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 46.68991459226145
MSE 2: 45.7301288839297
MSE 3: 34.63974633571785
MSE 4: 31.23384209666823
MSE 5: 31.07259714564845
MSE 6: 30.48245040498505
MSE 7: 33.56687707499779
MSE 8: 27.943381047171684
MSE 9: 28.109054454322
MSE 10: 26.181437705327006
MSE 11: 28.281916920806985
MSE 12: 24.19609306853952
MSE 13: 27.54699670612619
MSE 14: 30.782694634496202
MSE 15: 28.586960128981705
MSE 16: 21.527720392714812
MSE 17: 23.79693920172534
MSE 18: 25.7347662410304
MSE 19: 22.47182938967708
MSE 20: 28.04763413710116
MSE 21: 25.026463036398287
MSE 22: 22.094117383740866
MSE 23: 21.31919765472412
MSE 24: 23.101906699270106
MSE 25: 25.209764949712167
MSE 26: 25.971104075607744
MSE 27: 22.191890466560437
MSE 28: 24.163542553059106
MSE 29: 27.012678498203314
MSE 30: 21.132452807380158
MSE 31: 23.470134265986076
MSE 32: 19.599982008579094
MSE 33: 18.940307654223396
MSE 34: 21.59338493717527
MSE 35: 22.94854282021137
MSE 36: 26.445476673953355
MSE 37: 16.984663342966616
MSE 38: 21.106059324394153
MSE 